```python
[
    {"title": "CoT Infrastructure & Sentence Taxonomy", "icon": "1-circle-fill", "subtitle": "(30%)"},
    {"title": "Black-box Analysis", "icon": "2-circle-fill", "subtitle": "(30%)"},
    {"title": "White-box Methods", "icon": "3-circle-fill", "subtitle": "(40%)"},
]
```

# [1.6.5] LLM Psychology & Persona Vectors

<img src="https://raw.githubusercontent.com/info-arena/ARENA_img/refs/heads/main/img/header-65.png" width="350">

*Note - this content is subject to change depending on how much Anthropic publish about their [soul doc](https://simonwillison.net/2025/Dec/2/claude-soul-document/) over the coming weeks.*

# Introduction

Most exercises in this chapter have dealt with LLMs at quite a low level of abstraction; as mechanisms to perform certain tasks (e.g. indirect object identification, in-context antonym learning, or algorithmic tasks like predicting legal Othello moves). However, if we want to study the characteristics of current LLMs which might have alignment relevance, we need to use a higher level of abstraction. LLMs often exhibit "personas" that can shift unexpectedly - sometimes dramatically (see Sydney, Grok's "MechaHitler" persona, or [Tim Hua's work](https://www.lesswrong.com/posts/iGF7YcnQkEbwvYLPA/ai-induced-psychosis-a-shallow-investigation) on AI-induced psychosis). These personalities are clearly shaped through training and prompting, but exactly why remains a mystery.

In this section, we'll explore one approach for studying these kinds of LLM behaviours - **model psychiatry**. This sits at the intersection of evals (behavioural observation) and mechanistic interpretability (understanding internal representations / mechanisms). We aim to use interp tools to understand & intervene on behavioural traits.

The main focus will be on two different papers from Anthropic. First, we'll replicate the results from [The assistant axis: situating and stabilizing the character of large language models](https://www.anthropic.com/research/assistant-axis), which studies the "persona space" in internal model activations, and situates the "Assistant persona" within that space. The paper also introduces a method called **activation capping**, which identifies the normal range of activation intensity along this "Assistant Axis" and caps the model's activations when it would otherwise exceed it, which reduces the model's susceptibility to persona-based jailbreaks. Then, we'll move to the paper [Persona vectors: Monitoring and controlling character traits in language models](https://www.anthropic.com/research/persona-vectors) which predates the Assistant Axis paper but is broader and more methodologically sophisticated, proposing an automated pipeline for identifying persona vectors corresponding to specific kinds of undesireable personality shifts.

This section is (compared to many others in this chapter) very recent work, and there are still many uncertainties and unanswered questions! We'll suggest several bonus exercises or areas for further reading / exploration as we move through these exercises.

## Content & Learning Objectives

### 1️⃣ Mapping Persona Space

You'll start by understanding the core methodology from the [Assistant Axis](https://www.anthropic.com/research/assistant-axis) paper. You'll load Gemma 27b with activation caching utilities, and extract vectors corresponding to several different personas spanning from "helpful" to "fantastical".

> ##### Learning Objectives
> 
> * Understand the persona space mapping explored by the Assistant Axis paper
> * Given a persona name, generate a system prompt and collect responses to a diverse set of questions, to extract a mean activation vector for that persona
> * Briefly study the geometry of these persona vectors using PCA and cosine similarity

### 2️⃣ Steering along the Assistant Axis

Now that you've extracted these persona vectors, you should be able to use the Assistant Axis to detect drift and intervene via **activation capping**. As case studies, we'll use some of the dialogues saved out by Tim Hua in his investigation of AI-induced psychosis (link to GitHub repo [here](https://github.com/tim-hua-01/ai-psychosis)). By the end of this section, you should be able to steer to mitigate these personality shifts without kneecapping model capabilities.

> ##### Learning Objectives
> 
> * Steer towards directions you found in the previous section, to increase model willingness to adopt alternative personas
> * Understand how to use the Assistant Axis to detect drift and intervene via **activation capping**
> * Apply this technique to mitigate personality shifts in AI models (measuring the harmful response rate with / without capping)

### 3️⃣ Contrastive Prompting

Here, we move onto the [Persona Vectors](https://www.anthropic.com/research/persona-vectors) paper. You'll move from the global persona structure to surgical trait-specific vectors, exploring how to extract these vectors using contrastive prompt pairs.

> ##### Learning Objectives
> 
> * Understand the automated artifact pipeline for extracting persona vectors using contrastive prompts
> * Implement this pipeline (including autoraters for trait scoring) to extract "sycophancy" steering vectors
> * Learn how to identify the best layers trait extration
> * Interpret these sycophancy vectors using Gemma sparse autoencoders

### 4️⃣ Steering with Persona Vectors

Finally, you'll validate your extracted trait vectors through steering as well as projection-based monitoring. 

> ##### Learning Objectives
> 
> * Complete your artifact pipeline by implementing persona steering
> * Repeat this full pipeline for "hallucination" and "evil", as well as for any additional traits you choose to study
> * Study the geometry of trait vectors


## Setup code

In [ ]:
import os

# 1️⃣ Mapping Persona Space

First text

# 2️⃣ Steering along the Assistant Axis

First text

# 3️⃣ Contrastive Prompting

First text

# 4️⃣ Steering with Persona Vectors

First text

## ☆ Bonus

???